<a href="https://colab.research.google.com/github/alkarps/GB.Methods-for-collecting-and-processing-data-from-the-Internet/blob/hw03/hw03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install bs4
#!pip install requests

In [3]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
from time import sleep
import requests
import json

In [4]:
def saveJsonTo(file_name, income):
  json_string = json.dumps(income)
  with open(file_name, 'w') as outfile:
      outfile.write(json_string)

def readAndPrintJsonFrom(file_name):
  with open(file_name) as json_file:
    data = json.load(json_file)
    print(data)

In [5]:
class Vacancy:
    def __init__(self, source, name, link, min_salary = None, max_salary = None):
        self.source = source
        self.name = name
        self.link = link
        self.min_salary = min_salary
        self.max_salary = max_salary
    def __str__(self):
        return f"Vacancy(source={self.source}; name={self.name}; link={self.link}; min_salary={self.min_salary}; max_salary={self.max_salary})"

class Salary:
    def __init__(self, amount, currency):
        self.amount = amount
        self.currency = currency
    def __str__(self):
        return f"Salary(amount={self.amount}; currency={self.currency})"

In [6]:
def get_soup_by_url(url, payload):
  res = requests.get(url, params=payload)
  return bs(res.content, 'html.parser')

def get_hh_ru_soup(vacancy_name, page):
  return get_soup_by_url("https://hh.ru/search/vacancy", {"text": vacancy_name, 'page': page, 'hhtmFrom': 'vacancy_search_list'})

**Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru, SuperJob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Полученный список должен содержать в себе минимум:**



*   Наименование вакансии.
*   Предполагаемую зарплату (**дополнительно**: разносим на три поля: минимульную, максимальную и валюту. Цифры приводим к числовому формату). 
*   Ссылку на саму вакансию.
*   Сайт, откуда собрана вакансия.


По желанию, можно добавить еще параметров (например, работодатели в предложении). Структура должна быть одинакова для ванаксий со всех сайтов. Общий результат можно вывести в помощью dataFrame, сохранить в json или csv формат.

In [7]:
def parse_hh_page(soup):
  result = list()
  source = "hh.ru"
  vacancy_cards = soup.find_all("div", {"class": "vacancy-serp-item-body__main-info"})
  for vacancy_card in vacancy_cards:
    vacancy = vacancy_card.find("a", {"class": "serp-item__title"})
    link = vacancy["href"]
    title = vacancy.string
    salary = vacancy_card.find("span",{"class": "bloko-header-section-3"})
    min_salary = None
    max_salary = None
    if salary: 
      salary = salary.string.split("<!-- -->")
      if salary.startswith("от"):
        min_salary = Salary(salary[1].split(), salary[3])
      elif salary.startswith("до"):
        max_salary = Salary(salary[1].split(), salary[3])
      else:
        currency = salary[1]
        min_salary = Salary(salary[0].split(" - ")[0].split(), currency)
        max_salary = Salary(salary[0].split(" - ")[1].split(), currency)
    result.append(Vacancy(source, title, link, min_salary, max_salary))
  return result

In [8]:
def parse_hh(vacancy_name):
  result = list()
  for i in range(40):
    soup = get_hh_ru_soup(vacancy_name, str(i))
    vacancies = parse_hh_page(soup)
    result.extend(vacancies)
  return result

In [9]:
vanacies = parse_hh("тестер")
saveJsonTo("parsed_vacancies.json", vanacies)
readAndPrintJsonFrom("parsed_vacancies.json")

[]
